In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [2]:
train_df = pd.read_csv('./titanicTrain.csv')
test_df = pd.read_csv('./titanicQuestion.csv')

In [3]:
train_df = train_df[:1000]
print('Train:', len(train_df),
      'Test:', len(test_df))

Train: 1000 Test: 309


## preprocessing function

In [4]:
# Titanic data preprocessing function
def preprocessTitanicData(raw_df):
    df = raw_df[['survived', 'pclass', 'sibsp', 'parch', 'age', 'sex', 'fare', 'embarked']]
    df['age'] = df['age'].fillna(df['age'].mean())
    df['fare'] = df['fare'].fillna(df['fare'].mean())
    df['sex'] = df['sex'].map({'female':0, 'male':1}).astype(int)
    x_onehot_df = pd.get_dummies(data=df, columns= ['embarked'])
    ndarray = x_onehot_df.values
    feature = ndarray[:,1:]
    label = ndarray[:, 0]
    
    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
    scaledFeatures = minmax_scale.fit_transform(feature)
    
    return scaledFeatures, label

In [20]:
train_df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,prediction
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",1
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",1
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",1
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON",0
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",1


In [5]:
x_train, y_train = preprocessTitanicData(train_df)
x_test, y_test = preprocessTitanicData(test_df)

D:\andyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\andyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\andyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [6]:
print(x_train.shape, y_train.shape) #np.array np.array
print(x_test.shape, y_test.shape) # np.array np.array

# reshape y_train and y_test
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

print(x_train.shape, y_train.shape) #np.array np.array
print(x_test.shape, y_test.shape) # np.array np.array

(1000, 9) (1000,)
(309, 9) (309,)
(1000, 9) (1000, 1)
(309, 9) (309, 1)


## construct model

In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.transforms as transforms

In [8]:
class LR(nn.Module): # 繼承torch.nn.module特性
    
    # 架構
    def __init__(self):
        super(LR, self).__init__() # super(LR, self) 繼承自己的__init__()
        self.hidden1 = nn.Linear(9, 100) # input 9, hidden 40  Linear全連接
        self.dropout = nn.Dropout(0.1)
        self.hidden2 = nn.Linear(100, 80) # input 40, hidden 40  Linear全連接
        self.hidden3 = nn.Linear(80, 60) # input 40, hidden 40  Linear全連接
        self.hidden4 = nn.Linear(60, 10) # input 40, hidden 30  Linear全連接
        self.hidden5 = nn.Linear(10, 1) # hidden 10, output 1

    # 運作流程（資料流）
    def forward(self, x):  
        x = F.relu(self.hidden1(x)) # 讓x由hidden1後，進入activation function
        x = F.dropout(self.dropout(x))
        x = F.relu(self.hidden2(x))
        x = F.dropout(self.dropout(x))
        x = F.relu(self.hidden3(x))
        x = F.dropout(self.dropout(x))
        x = F.relu(self.hidden4(x))
        out = self.hidden5(x)
        return out

In [9]:
model = LR()
loss_func = nn.MSELoss()
learning_rate = 0.03
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) #model.parameters() 所有訓練的參數都在這
# model.cuda()   

## Training

In [10]:
epochs = 5000

In [11]:
for epoch in range(epochs):
    epoch += 1

    # convert to variables
    x = Variable(torch.from_numpy(x_train).float())
    y = Variable(torch.from_numpy(y_train).float())

    # clear gradient w.r.t. parameters 
    optimizer.zero_grad()
    
    # forward to get output
    prediction = model(x)

    # calculate loss
    loss = loss_func(prediction, y)

    # backward to get gradient
    loss.backward()

    # update parameters
    optimizer.step() 

    if epoch % 5 == 0:
    # plot and show learning process
        print("epoch %d, loss %.8f" % (epoch, loss.data[0]))

D:\andyl\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 5, loss 0.24885741
epoch 10, loss 0.24728897
epoch 15, loss 0.24634445
epoch 20, loss 0.24545364
epoch 25, loss 0.24433812
epoch 30, loss 0.24333763
epoch 35, loss 0.24348482
epoch 40, loss 0.24297087
epoch 45, loss 0.24280015
epoch 50, loss 0.24146870
epoch 55, loss 0.24090330
epoch 60, loss 0.24094777
epoch 65, loss 0.24064083
epoch 70, loss 0.23940931
epoch 75, loss 0.23890696
epoch 80, loss 0.23885879
epoch 85, loss 0.23778822
epoch 90, loss 0.23831534
epoch 95, loss 0.23704524
epoch 100, loss 0.23616745
epoch 105, loss 0.23681906
epoch 110, loss 0.23548934
epoch 115, loss 0.23471224
epoch 120, loss 0.23440093
epoch 125, loss 0.23370431
epoch 130, loss 0.23258600
epoch 135, loss 0.23138948
epoch 140, loss 0.23095873
epoch 145, loss 0.23064688
epoch 150, loss 0.22994404
epoch 155, loss 0.22860654
epoch 160, loss 0.22820181
epoch 165, loss 0.22649284
epoch 170, loss 0.22588585
epoch 175, loss 0.22454563
epoch 180, loss 0.22437683
epoch 185, loss 0.22105432
epoch 190, loss 0.221

epoch 1520, loss 0.14690433
epoch 1525, loss 0.14389047
epoch 1530, loss 0.14904706
epoch 1535, loss 0.14772342
epoch 1540, loss 0.14747240
epoch 1545, loss 0.14675112
epoch 1550, loss 0.14669217
epoch 1555, loss 0.14382799
epoch 1560, loss 0.14523979
epoch 1565, loss 0.14597531
epoch 1570, loss 0.14965121
epoch 1575, loss 0.14295098
epoch 1580, loss 0.14892979
epoch 1585, loss 0.14762829
epoch 1590, loss 0.14631128
epoch 1595, loss 0.14643760
epoch 1600, loss 0.14512511
epoch 1605, loss 0.14583069
epoch 1610, loss 0.14560260
epoch 1615, loss 0.14340295
epoch 1620, loss 0.14584443
epoch 1625, loss 0.14641370
epoch 1630, loss 0.14538915
epoch 1635, loss 0.14608912
epoch 1640, loss 0.14264214
epoch 1645, loss 0.14536916
epoch 1650, loss 0.14549915
epoch 1655, loss 0.14615126
epoch 1660, loss 0.14456627
epoch 1665, loss 0.14394104
epoch 1670, loss 0.14404991
epoch 1675, loss 0.14655448
epoch 1680, loss 0.14592576
epoch 1685, loss 0.14755568
epoch 1690, loss 0.14323728
epoch 1695, loss 0.1

epoch 2995, loss 0.14045502
epoch 3000, loss 0.14135467
epoch 3005, loss 0.13734104
epoch 3010, loss 0.14061810
epoch 3015, loss 0.13804662
epoch 3020, loss 0.13744850
epoch 3025, loss 0.14211977
epoch 3030, loss 0.14015947
epoch 3035, loss 0.14296870
epoch 3040, loss 0.14172627
epoch 3045, loss 0.14022714
epoch 3050, loss 0.14135398
epoch 3055, loss 0.14107506
epoch 3060, loss 0.14029799
epoch 3065, loss 0.14101687
epoch 3070, loss 0.13947849
epoch 3075, loss 0.13697425
epoch 3080, loss 0.14243007
epoch 3085, loss 0.14084774
epoch 3090, loss 0.14082350
epoch 3095, loss 0.14195444
epoch 3100, loss 0.13887095
epoch 3105, loss 0.13972117
epoch 3110, loss 0.13949549
epoch 3115, loss 0.14039876
epoch 3120, loss 0.13901065
epoch 3125, loss 0.13976261
epoch 3130, loss 0.14253224
epoch 3135, loss 0.14099160
epoch 3140, loss 0.14024606
epoch 3145, loss 0.14491765
epoch 3150, loss 0.13894507
epoch 3155, loss 0.14010148
epoch 3160, loss 0.13972141
epoch 3165, loss 0.13927835
epoch 3170, loss 0.1

epoch 4475, loss 0.13695878
epoch 4480, loss 0.13671857
epoch 4485, loss 0.13516836
epoch 4490, loss 0.13606933
epoch 4495, loss 0.13611986
epoch 4500, loss 0.13695683
epoch 4505, loss 0.13799810
epoch 4510, loss 0.13617483
epoch 4515, loss 0.13708024
epoch 4520, loss 0.13926543
epoch 4525, loss 0.13863835
epoch 4530, loss 0.13706943
epoch 4535, loss 0.13564686
epoch 4540, loss 0.13795170
epoch 4545, loss 0.13422205
epoch 4550, loss 0.13665441
epoch 4555, loss 0.13724245
epoch 4560, loss 0.13616832
epoch 4565, loss 0.13354589
epoch 4570, loss 0.13709152
epoch 4575, loss 0.13582423
epoch 4580, loss 0.13594900
epoch 4585, loss 0.13675806
epoch 4590, loss 0.13662037
epoch 4595, loss 0.13786319
epoch 4600, loss 0.13684276
epoch 4605, loss 0.13951176
epoch 4610, loss 0.14063644
epoch 4615, loss 0.13607837
epoch 4620, loss 0.13623859
epoch 4625, loss 0.13672094
epoch 4630, loss 0.13618028
epoch 4635, loss 0.13748902
epoch 4640, loss 0.13624553
epoch 4645, loss 0.13500866
epoch 4650, loss 0.1

In [12]:
# get filters from parameters:
print(list(model.parameters())[0].size())  

torch.Size([100, 9])


In [13]:
train_prediction = prediction.data.numpy()
train_prediction = np.where(train_prediction>=0.7,1,0)
train_df['prediction'] = train_prediction

## Prediction

In [14]:
x_test_v = Variable(torch.from_numpy(x_test).float())
test_prediction = model(x_test_v).data.numpy()
# test_prediction = np.where(test_prediction>=0.6,1,0)

In [15]:
test_df['prediction'] = test_prediction

In [16]:
def apply_sub(row, label):
    if row[label] >= 0.5:
        return 1
    else:
        return 0

test_df['prediction'] = test_df.apply(lambda x: apply_sub(x, 'prediction'), axis = 1)
test_df['prediction']

0      0
1      1
2      1
3      0
4      1
5      0
6      1
7      1
8      1
9      0
10     0
11     1
12     1
13     1
14     1
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     1
24     0
25     1
26     1
27     0
28     1
29     0
      ..
279    1
280    0
281    0
282    0
283    0
284    0
285    0
286    1
287    0
288    0
289    0
290    1
291    0
292    0
293    0
294    0
295    0
296    0
297    0
298    0
299    0
300    1
301    0
302    1
303    0
304    1
305    1
306    0
307    0
308    0
Name: prediction, Length: 309, dtype: int64

In [17]:
#test_df['prediction'].to_csv('titanic_answer_trained.csv')

## Verify model

In [21]:
answer = pd.read_csv('titanicAns.csv')
print('Accuracy: %f %%'%(
    (1.0 - sum(abs(answer['survived'] - test_df['prediction'])) / test_df.shape[0]) * 100)
)

Accuracy: 69.902913 %
